# Run a basic Hermes Hartree-Fock energy calculation on ligand conformers from Auto3d

In this notebook, we'll perform a hermes energy calculation on a ligand derived from a smiles string

# 0) Complete Example
See the [sample notebook](/Quickstarts/auto3d_conformer_tautomer_generation-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [1]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

## 1.1) Configuration

In [2]:
EXPERIMENT = "tengu-py-auto3d_hermes"
LIGAND_SMILES = "CC(=O)OC1=CC=CC=C1C(=O)O "
LIGAND = "ASPRIN"
TAGS = ["qdx", EXPERIMENT, LIGAND]

In [3]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys

os.chdir(WORK_DIR)

## 1.2) Build your client

In [4]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(batch_tags=TAGS)

In [5]:
# |hide
client = await rush.build_provider_with_functions(batch_tags=TAGS, restore_by_default=True)

In [6]:
# |hide
client.workspace = WORK_DIR
client.config_dir = WORK_DIR / ".rush"

# 2) Preparation
We want to convert our raw files into prepared qdxfs, with correct charges and missing residues filled

In [7]:
help(client.auto3d)

Help on function auto3d in module rush.provider:

async auto3d(*args: *tuple[RushObject[bytes], str, Record], target: Optional[Target] = 'NIX_SSH', resources: Optional[Resources] = {'storage': 18, 'storage_units': 'MB', 'gpus': 1}, tags: list[str] | None = None, restore: bool | None = None) -> tuple[RushObject[bytes], RushObject[list[Conformer]]]
    Generate 3D conformers from SMILES strings and other inputs
    
    Module version:  
    `github:talo/tengu-auto3d/f5f26308f74bde570c870ff0b5bb1be42ac9c712#auto3d_tengu`
    
    QDX Type Description:
    
        molecule_file: @bytes;
        molecule_file_type: string;
        options: {
            capacity: u32?,
            convergence_threshold: f32?,
            enumerate_isomer: bool?,
            enumerate_tautomer: bool?,
            gpu_idx: [u32]?,
            job_name: string?,
            k: i32?,
            max_confs: u32?,
            memory: u32?,
            mpi_np: u32?,
            opt_steps: u32?,
            optim

In [ ]:
ligand_path = client.workspace / "aspirin.smiles"
with open(ligand_path, "w") as ligand_file:
    print(f"{LIGAND_SMILES} {LIGAND_SMILES}", file=ligand_file)

(conformer_sdf, conformer_qdxf) = await client.auto3d(
    ligand_path, "smi", {"k": 5, "use_gpu": True}, resources={"gpus": 1, "storage": "5", "storage_units": "GB"},
    target="NIX_SSH_3"
)

print(f"{datetime.now().time()} | Running ligand preparation!")

In [9]:
try:
    await conformer_sdf.download(filename="01_prepared_ligand.sdf")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file; you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

You should visualize your prepared ligand to spot check any issues

In [10]:
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_ligand.sdf", "r") as f:
    view.addModel(f.read(), "sdf")
    view.setStyle({"stick": {}})
    view.zoomTo()
    view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# 3) Quantum energy calculation
Finally, we submit our fragmented protein for quantum energy calculation, with custom configuration.

In [11]:
help(client.hermes_energy)

Help on function hermes_energy in module rush.provider:

async hermes_energy(*args: *tuple[RushObject[Conformer], Record, Optional[Record], Optional[Record]], target: Optional[Target] = 'NIX_SSH', resources: Optional[Resources] = {'storage': 1034, 'storage_units': 'MB', 'gpus': 4}, tags: list[str] | None = None, restore: bool | None = None) -> tuple[RushObject[Record]]
    Runs a HERMES energy calculation given a topology, and optionally model and keyword configurations.
    Will use the default model and keywords if none are provided
    
    Module version:  
    `github:talo/tengu-prelude/e94a09ac64990a91cf1fcd33858d565a3d33ba09#hermes_energy`
    
    QDX Type Description:
    
        input: @Conformer;
        system: {
            gpus_per_team: u32?,
            max_gpu_memory_mb: u32?,
            oversubscribe_gpus: bool?,
            teams_per_node: u32?
        };
        model: {
            aux_basis: string?,
            basis: string,
            force_cartesian_basis_s

In [12]:
HERMES_RESOURCES = {
        "gpus": 4,
        "storage": 100,
        "storage_units": "MB",
        "walltime": 60 * 4,
}

In [ ]:
(conformer,) = await client.pick_conformer(conformer_qdxf, 0)
conformer_out = await conformer.get()

(hermes_energy,) = await client.hermes_energy(
    conformer,
    {},
    {
        "method": "RestrictedRIMP2",
        "basis": "cc-pVDZ",
        "aux_basis": "cc-pVDZ-RIFIT",
        "force_cartesian_basis_sets": True,
        "standard_orientation": "FullSystem",
    },
    {
    "scf": {
        "max_iter": 50,
        "max_diis_history_length": 12,
        "convergence_metric": "DIIS",
        "convergence_threshold": 0.000001,
        "density_threshold": 1e-10,
        "density_basis_set_projection_fallback_enabled": True,
        },
    "frag": {
        "cutoffs": {"dimer": 22},
        "cutoff_type": "Centroid",
        "level": "Dimer",
        "reference_fragment": len(conformer_out["topology"]["fragments"])
        - 1,
        },
    },
    target="GADI",
    resources=HERMES_RESOURCES,
)

In [14]:
await hermes_energy.get()

{'distance_metric': 'Centroid',
 'expanded_density': None,
 'expanded_hf_energy': 0.0,
 'expanded_hf_gradients': None,
 'expanded_mp2_gradients': None,
 'expanded_mp2_os_correction': 0.0,
 'expanded_mp2_ss_correction': 0.0,
 'method': 'RestrictedRIMP2',
 'nmers': [[{'bond_orders': None,
    'coeffs_final': None,
    'coeffs_initial': None,
    'delta_hf_energy': None,
    'delta_mp2_os_correction': None,
    'delta_mp2_ss_correction': None,
    'density': None,
    'fock': None,
    'fragment_distance': None,
    'fragments': [0],
    'h_caps': None,
    'h_core': None,
    'hf_energy': -645.0201843607325,
    'hf_gradients': None,
    'molecular_orbital_energies': None,
    'mp2_gradients': None,
    'mp2_os_correction': -1.432208203979463,
    'mp2_ss_correction': -0.5339044712553163,
    'mulliken_charges': None,
    'num_basis_fns': 235,
    'num_iters': 9,
    'overlap': None}]],
 'num_iters': 9,
 'reference_fragment': 0}